# Face Detection

This notebook is used for quick tests

In [ ]:
%load_ext autoreload
%autoreload 2

# put system imports here
import os
import sys
from os.path import join
from glob import glob
from pathlib import Path

# put pip/conda installed imports here
import cv2 as cv
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
import random

# put local imports here
fp_app_root = join(Path(os.getcwd()).parent)
sys.path.append(fp_app_root)
from app.settings import app_cfg
from app.utils import file_utils, draw_utils
from app.models.bbox import BBoxNorm

## Load image

In [ ]:
filepaths = glob(join(app_cfg.DIR_IMAGES, '*.jpg'))
fp_im = filepaths[random.randint(0, len(filepaths))]
im_bgr = cv.imread(fp_im)

In [ ]:
# Define a function to detect faces using OpenCV's haarcascades
def detect_faces_haar(classifier, im, scale_factor=1.1,overlaps=3,min_size=70, max_size=700,flags=0):
    
  min_size = (min_size, min_size) # minimum face size
  max_size = (max_size, max_size) # maximum face size

  # Convert to grayscale
  im_gray = cv.cvtColor(im, cv.COLOR_BGR2GRAY)

  # Run detector
  #matches = classifier.detectMultiScale(im_gray, scale_factor, overlaps, flags, min_size, max_size)
  matches = classifier.detectMultiScale(im_gray, 
                                        scaleFactor=scale_factor, 
                                        minNeighbors=overlaps, 
                                        minSize=min_size,
                                        flags=cv.CASCADE_SCALE_IMAGE)
  # By default, OpenCV returns x,y,w,w
  # Modify to return x1,y1,x2,y2 (more convenient for drawing)
  matches = [ (r[0],r[1],r[0]+r[2],r[1]+r[3]) for r in matches]

  return matches

In [ ]:
# Load cascades
fp_haarcascade = app_cfg.FP_FRONTALFACE
classifier = cv.CascadeClassifier(fp_haarcascade)

In [ ]:
matches = detect_faces_haar(classifier, im_bgr, min_size=40, max_size=200)
print(matches)

## Draw face

In [ ]:
im_dst = im_bgr.copy()

for match in matches:
  x,y,w,h = match[0], match[1], match[2] - match[0], match[3] - match[1]
  im_dst = cv.rectangle(im_dst, (x,y,w,h), (0,255,0), 4, cv.LINE_AA)
  
plt.imshow(cv.cvtColor(im_dst, cv.COLOR_BGR2RGB))

In [ ]:
# we can do better

im_dst = im_bgr.copy()
matches = detect_faces_haar(classifier, im_bgr, min_size=40, overlaps=5)
for match in matches:
  x,y,w,h = match[0], match[1], match[2] - match[0], match[3] - match[1]
  im_dst = cv.rectangle(im_dst, (x,y,w,h), (0,255,0), 4, cv.LINE_AA)
  
plt.imshow(cv.cvtColor(im_dst, cv.COLOR_BGR2RGB))

In [ ]:
# we can do worse

im_dst = im_bgr.copy()
matches = detect_faces_haar(classifier, im_bgr, min_size=40, overlaps=0)
for match in matches:
  x,y,w,h = match[0], match[1], match[2] - match[0], match[3] - match[1]
  im_dst = cv.rectangle(im_dst, (x,y,w,h), (0,255,0), 2, cv.LINE_AA)
  
plt.imshow(cv.cvtColor(im_dst, cv.COLOR_BGR2RGB))